In [19]:
import pandas as pd
import os
import numpy as np
os.chdir(r'C:\Users\REGGIE\Desktop\work')

In [20]:
train=pd.read_csv('train.csv')
train.head()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,4,1,0,0,1,6,1,5,0,0,...,0,1,2,0,0,0,0,0,0,Class_1
4,5,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


In [21]:
y_train=train['target']
X_train=train.drop(['id','target'],axis=1)
#保存特征名字以备后用（可视化）
feat_names=X_train.columns

#sklearn的学习器大多支持稀疏矩阵数据输入，模型训练会快很多
#查看一个学习器是否支持稀疏数据，可以看fit函数是否支持：X:{array-like,
# sparse matrix}

from scipy.sparse import csr_matrix
X_train=csr_matrix(X_train)

In [22]:
train.target.value_counts()

Class_2    16122
Class_6    14135
Class_8     8464
Class_3     8004
Class_9     4955
Class_7     2839
Class_5     2739
Class_4     2691
Class_1     1929
Name: target, dtype: int64

In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

def grid_search(X_train,y_train):
    #需要调优的参数
    #请尝试将L1正则和L2正则分开，并配合合适的优化求解算法（solver）
    #tuned_parameters={'penalth':['l1','l2'],'C':[0.001,0.01,0.1,1,10,100,1000]}
    
    #参数的搜索范围
    penaltys=['l1','l2']
    Cs=[0.1,1,10,100,1000]
    
    #调优的参数集合，搜索网格为2x5，在网格上的交叉点进行搜索
    tuned_parameters=dict(penalty=penaltys,C=Cs)


    lr_penalty=LogisticRegression(solver='liblinear')
    grid=GridSearchCV(lr_penalty,tuned_parameters,cv=3,scoring='neg_log_loss',n_jobs=4)

    model = grid.fit(X_train,y_train)
    best_model = grid.best_params_
    
    return model,best_model

In [24]:
grid_search(X_train,y_train)

(GridSearchCV(cv=3, error_score=nan,
              estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                           fit_intercept=True,
                                           intercept_scaling=1, l1_ratio=None,
                                           max_iter=100, multi_class='auto',
                                           n_jobs=None, penalty='l2',
                                           random_state=None, solver='liblinear',
                                           tol=0.0001, verbose=0,
                                           warm_start=False),
              iid='deprecated', n_jobs=4,
              param_grid={'C': [0.1, 1, 10, 100, 1000], 'penalty': ['l1', 'l2']},
              pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
              scoring='neg_log_loss', verbose=0), {'C': 1, 'penalty': 'l2'})

In [7]:
#pyspark中使用逻辑回归+网格搜索
train_spark = train

In [8]:
train_spark.replace('Class_1' , 1, inplace = True)
train_spark.replace('Class_2' , 2 , inplace = True)
train_spark.replace('Class_3' , 3 , inplace = True)
train_spark.replace('Class_4' , 4 , inplace = True)
train_spark.replace('Class_5' , 5 , inplace = True)
train_spark.replace('Class_6' , 6 , inplace = True)
train_spark.replace('Class_7' , 7 , inplace = True)
train_spark.replace('Class_8' , 8 , inplace = True)
train_spark.replace('Class_9' , 9 , inplace = True)

In [9]:
train_spark.head(7)

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,4,1,0,0,1,6,1,5,0,0,...,0,1,2,0,0,0,0,0,0,1
4,5,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
5,6,2,1,0,0,7,0,0,0,0,...,0,3,0,0,0,0,2,0,0,1
6,7,2,0,0,0,0,0,0,2,0,...,1,1,0,0,0,0,0,0,1,1


In [10]:
train_spark.target.value_counts()

2    16122
6    14135
8     8464
3     8004
9     4955
7     2839
5     2739
4     2691
1     1929
Name: target, dtype: int64

In [11]:
#创建SparkSession
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext
#%%
#先转化为spark——df
sc = SparkContext.getOrCreate();
sqlContest = SQLContext(sc)
df = sqlContest.createDataFrame(train_spark)

In [12]:
#特征处理
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.linalg import Vectors
#注意X从1开始，去除不需要的ID列
data=df.rdd.map(lambda x:(Vectors.dense(x[1:-1]), x[-1])).toDF(["features", "label"])
data.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[1.0,0.0,0.0,0.0,...|    1|
|[0.0,0.0,0.0,0.0,...|    1|
|[0.0,0.0,0.0,0.0,...|    1|
|[1.0,0.0,0.0,1.0,...|    1|
|[0.0,0.0,0.0,0.0,...|    1|
|[2.0,1.0,0.0,0.0,...|    1|
|[2.0,0.0,0.0,0.0,...|    1|
|[0.0,0.0,0.0,0.0,...|    1|
|[0.0,0.0,0.0,0.0,...|    1|
|[0.0,0.0,0.0,0.0,...|    1|
|[0.0,1.0,1.0,2.0,...|    1|
|[0.0,1.0,2.0,1.0,...|    1|
|[1.0,0.0,1.0,0.0,...|    1|
|[0.0,0.0,0.0,1.0,...|    1|
|[0.0,0.0,0.0,0.0,...|    1|
|[0.0,0.0,0.0,2.0,...|    1|
|[0.0,0.0,0.0,0.0,...|    1|
|[0.0,0.0,0.0,0.0,...|    1|
|[0.0,0.0,0.0,0.0,...|    1|
|[0.0,0.0,0.0,0.0,...|    1|
+--------------------+-----+
only showing top 20 rows



In [27]:
# spark超参调优
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.ml.classification import LogisticRegression
# 评估模型性能
import pyspark.ml.evaluation as ev

In [28]:
def spark_search(data):
    #使用网格搜索的逻辑回归
    logistic = LogisticRegression(labelCol='label')

    paramGrid = ParamGridBuilder().addGrid(logistic.maxIter,[2,10,20,30]).addGrid(logistic.regParam, [0.01, 0.05, 0.08, 0.1]).build()

    evaluator = ev.BinaryClassificationEvaluator(
        rawPredictionCol='probability',labelCol='label')
    
    #网格搜索
    cv = CrossValidator(estimator=logistic,
                              estimatorParamMaps=paramGrid,
                              evaluator=evaluator,
                              numFolds=2)
    cvModel = cv.fit(data)
    bestModel = cvModel.bestModel
    bestparam = cvModel.getEstimatorParamMaps()[np.argmax(cvModel.avgMetrics)]
    
    return bestparam

In [29]:
spark_search(data)

{Param(parent='LogisticRegression_87f822f841ea', name='maxIter', doc='max number of iterations (>= 0).'): 2,
 Param(parent='LogisticRegression_87f822f841ea', name='regParam', doc='regularization parameter (>= 0).'): 0.01}

In [17]:
# #模型摘要
# trainingSummary = bestModel.summary

In [18]:
# #模型重要指标
# accuracy = trainingSummary.accuracy
# falsePositiveRate = trainingSummary.weightedFalsePositiveRate
# truePositiveRate = trainingSummary.weightedTruePositiveRate
# fMeasure = trainingSummary.weightedFMeasure()
# precision = trainingSummary.weightedPrecision
# recall = trainingSummary.weightedRecall
# print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
#       % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))